In [ ]:
import pyfcctab
import copy
import matplotlib.pyplot as plt
import astropy.units as units

In [ ]:
%config InlineBackend.figure_format = 'retina'
# from astropy.visualization import quantity_support
# quantity_support()

In [ ]:
tables = pyfcctab.read()

In [ ]:
fixed = tables.itu.stitch(
    lambda band: band.has_allocation("Fixed*", but_not="Fixed-Satellite*", primary=True))
fixed.metadata["label"] = "Fixed"

eess = tables.itu.stitch(
    lambda band: band.has_allocation("Earth Exploration-Satellite (Passive)*"))
eess.metadata["label"] = "EESS (Passive)"

ras = tables.itu.stitch(
    lambda band: band.has_allocation("Radio Astronomy*"))
ras.metadata["label"] = "RAS"

aep = tables.itu.stitch(
    lambda band: band.has_footnote("5.340*"))
aep.metadata["label"] = "5.340"

In [ ]:
# dummy=pyfcctab.htmltable(eess)

In [ ]:
specific_allocations = ["Earth Exploration-Satellite*", "Radio Astronomy*"]
for fixed_band in fixed:
    overlapped_bands = []
    adjacent_bands = []
    for science_bands in [eess, ras]:
        for science_band in science_bands:
            if fixed_band.overlaps(science_band):
                overlapped_bands.append(science_band)
            if fixed_band.is_adjacent(science_band):
                adjacent_bands.append(science_band)
    if overlapped_bands or adjacent_bands:
        print (f"========================================")
        print (f"Fixed from {fixed_band.range_str()}")
        for collection, prefix in [
            [overlapped_bands, "Overlaps"],
            [adjacent_bands, "Is adjacent to"]]:
            for b in collection:
                print (f"{prefix} {b.compact_str(specific_allocations=specific_allocations, skip_annotations=True)}")

In [ ]:
# Fix the 5.340 at 48.2-50.2
aep_patched = copy.deepcopy(aep)
b = aep_patched[48.3*units.GHz][0]
b.bounds = [ 48.94*units.GHz, 49.04*units.GHz ]
extra = copy.deepcopy(b)
extra.bounds = [50.2*units.GHz, 50.4*units.GHz]
aep_patched.append(extra)
print (aep_patched[48.3*units.GHz])

In [ ]:
pyfcctab.plot_bands(fixed, eess, ras, aep_patched)
plt.savefig("ai91c.pdf")

In [ ]:
for b in aep:
    print ('==============')
    print (b)